## What is a sequence (Revisited)

* A **sequence** is a series of values that can be iterated using **for-each** loop
* The values may be 
    * stored or 
    * computed on the fly

#### Iterable

* alternative term for sequence
* something that has an iterator and can be iterated (one by one)


```python 
seq= MySequence()
i = iter(seq)
```

* should have special method
    *  **\_\_iter\_\_**

#### Iterator

* An object that helps you iterate a sequence
* It allows you to access next items
    * has a special method 
        * **\_\_next\_\_()**


* first call to next gets first item
* calling next after last raises StopIteration

```python 

next(i) #first item
next(i) #second item
... 
next(i) #last item

next(i) #raise Stop Iteration
```

### Working with built-in Sequence


In [3]:
values=[4,8,11]

i1= iter(values)
i2= iter(values)

print(type(i1),id(i1))
print(type(i2),id(i2))

<class 'list_iterator'> 1462198848528
<class 'list_iterator'> 1462198852080


In [4]:
next(i1)

4

In [5]:
print(next(i1))
print(next(i1))

8
11


In [6]:
print(next(i1))

StopIteration: 

### Internal working of for-each loop

* you can think of a foreach loop as a while loop under the hood



In [7]:
def for_each(seq):
    for value in seq:
        print(value,end='\t')
    print()

def for_each_internal(seq):
    it=iter(seq)
    try:
        while True:
            value=next(it)
            print(value,end='\t')
    except StopIteration:
        pass
    print()


In [8]:
values={2,3,9,2,1}

for_each(values)

for_each_internal(values)


9	2	3	1	
9	2	3	1	


### Creating a Userdefined Sequence

* we need a obj which can be used as **iter(obj)**
* it should contain a method \_\_iter\_\_()

In [9]:
class MySequence:
    pass

In [10]:
for value in MySequence():
    print(value)

TypeError: 'MySequence' object is not iterable

In [11]:
s=MySequence()
i=iter(s)

TypeError: 'MySequence' object is not iterable

#### Lets make our sequence Iterable

In [12]:
class MySequence:
    def __iter__(self):
        pass

In [13]:
s=MySequence()
for v in s:
    print(v)

TypeError: iter() returned non-iterator of type 'NoneType'

### iter should return an object that has \_\_next\_\_


#### simplest empty sequence

In [14]:
class MySequence:
    def __iter__(self):
        return MySequence.MyIterator()
    
    class MyIterator:
        def __next__(self):
            raise StopIteration()  



In [15]:
for v in MySequence():
    print(v)

print('end')

end


### Stored Sequence : FixedArray

In [18]:
class FixedArray:
    def __init__(self,size):
        self._array=[]
        for x in range(size):
            self._array.append(None)

    def __len__(self):
        return len(self._array)
    
    def get(self,index):
        return self._array[index]
    
    def set(self,index,value):
        self._array[index]=value

    def __str__(self):
        s=f'FixedArray({len(self)})[\t'
        for v in self._array:
            s+=f'{v}\t'
        s+="]"
        return s


In [19]:

f=FixedArray(10)
print(f)

FixedArray(10)[	None	None	None	None	None	None	None	None	None	None	]


In [20]:
f.set(2,10)
f.set(3,20)
print(f)

FixedArray(10)[	None	None	10	20	None	None	None	None	None	None	]


In [21]:
f.set(20,11)

IndexError: list assignment index out of range

In [23]:
f=FixedArray(5)
for x in range(5):
    f.set(x,x*10)
print(f)


FixedArray(5)[	0	10	20	30	40	]


#### can we use for-each loop with our Fixed Array?


In [24]:
for value in f:
    print(value)

TypeError: 'FixedArray' object is not iterable

### How to make fixed Array Iterable?

* we need a method called \_\_iter\_\_ in fixed Array
* it should reuturn an object that should have \_\_next\_\_ in it

In [32]:
class FixedArrayIterator:
    def __init__(self,fixed_array):
        self._fixed_array=fixed_array
        self._current_index=-1

    def __next__(self):
        self._current_index+=1
        if(self._current_index>=len(self._fixed_array)):
            raise StopIteration()
        else:
            return self._fixed_array._array[self._current_index]

def fixed_array_iter(fa):
    return FixedArrayIterator(fa)

FixedArray.__iter__=fixed_array_iter

In [33]:
fa=FixedArray(10)
for x in range(10):
    fa.set(x,x*10)

print(fa)

FixedArray(10)[	0	10	20	30	40	50	60	70	80	90	]


In [34]:
for value in fa:
    print(value,end='\t')

0	10	20	30	40	50	60	70	80	90	

#### Implementing indexers

* we can make fixed array look just like standard array by making it work with []
* by default it doesn't work

In [35]:
fa=FixedArray(5)
fa[2]

TypeError: 'FixedArray' object is not subscriptable

#### Adding subscript to access the value

In [48]:
def get_item(array,index):
    return array._array[index]


FixedArray.__getitem__= get_item

In [49]:
fa=FixedArray(5)
print(fa[2])

None


#### But \_\_getitem\_\_ doesn't allow modification of value

In [50]:
fa=FixedArray(5)
fa[0]='Hi'

TypeError: 'FixedArray' object does not support item assignment

### This can be done using \_\_setitem\_\_

In [51]:
def set_item(array,index,value):
    array._array[index]=value

FixedArray.__setitem__=set_item

In [53]:
fa=FixedArray(10)
for i in range(len(fa)):
    fa[i]=i*10

for v in fa:
    print(v,end='\t')

print()
print(fa)

0	10	20	30	40	50	60	70	80	90	
FixedArray(10)[	0	10	20	30	40	50	60	70	80	90	]


### Complete Code of Fixed Array

In [59]:
#from math import min

class FixedArray:
    def __init__(self,size,*args):
        self._array=[]
        for x in range(size):
            self._array.append(None)
        for x in range(min(size,len(args))):
            self._array[x]=args[x]
        

    def __len__(self):
        return len(self._array)
    
    def __getitem__(self,index):
        return self._array[index]
    
    def __setitem__(self,index,value):
        self._array[index]=value

    def __str__(self):
        s=f'FixedArray({len(self)})[\t'
        for v in self._array:
            s+=f'{v}\t'
        s+="]"
        return s

    def __iter__(self):
        return FixedArray.Iterator(self)

    class Iterator:
        def __init__(self, array):
            self._array=array
            self._index=-1
        def __next__(self):
            self._index+=1
            if self._index>=len(self._array):
                raise StopIteration()
            else:
                return self._array[self._index]


In [60]:
fa=FixedArray(5,1,2,3,4)
print(fa)

FixedArray(5)[	1	2	3	4	None	]


In [61]:
for value in fa:
    print(value,end='\t')

1	2	3	4	None	

In [62]:
for i in range(len(fa)):
    print(fa[i],end='\t')

1	2	3	4	None	

### Computed Sequence

* sometimes we need a computed sequence
* the value is not stored anywhere but computed on the go
* example
    * prime_range(min,max)
    * fibnocii_series(10) #upto 10 items
    * random_numbers(1,6,100) # 100 random numbers between 1 and 6

#### generating random number sequence

* we can get random values using **random** module


#### let us understand random first

In [66]:
import random
print(dir(random))


['BPF', 'LOG4', 'NV_MAGICCONST', 'RECIP_BPF', 'Random', 'SG_MAGICCONST', 'SystemRandom', 'TWOPI', '_ONE', '_Sequence', '_Set', '__all__', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__', '_accumulate', '_acos', '_bisect', '_ceil', '_cos', '_e', '_exp', '_floor', '_index', '_inst', '_isfinite', '_log', '_os', '_pi', '_random', '_repeat', '_sha512', '_sin', '_sqrt', '_test', '_test_generator', '_urandom', '_warn', 'betavariate', 'choice', 'choices', 'expovariate', 'gammavariate', 'gauss', 'getrandbits', 'getstate', 'lognormvariate', 'normalvariate', 'paretovariate', 'randbytes', 'randint', 'random', 'randrange', 'sample', 'seed', 'setstate', 'shuffle', 'triangular', 'uniform', 'vonmisesvariate', 'weibullvariate']


In [68]:
help(random.randint)

Help on method randint in module random:

randint(a, b) method of random.Random instance
    Return random integer in range [a, b], including both end points.



In [71]:
random.randint(2,10)

6

### Let us now create a sequence of x values

#

In [72]:
from random import randint

class RandomValues:
    def __init__(self, min=0, max=100, total=None):
        self._min=min
        self._max=max
        self._total=total
        self._count=0

    def __iter__(self):
        return self # I am my own iterator. I have a __next__

    def __next__(self):
        self._count+=1
        if self._total!=None and self._count>self._total:
            raise StopIteration()
        else:
            return randint(self._min,self._max)

    

In [76]:
for number in RandomValues(1,6,20): #20 random numbers between 1 and 6
    print(number,end=' ')

3 6 5 2 5 3 2 1 3 1 3 3 2 3 5 2 3 2 3 4 

In [77]:
## returning infinite values
x=0
for number in RandomValues(1,6):
    x+=1
    if x%100==0:
        print('+',end=' ')
    

+ + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + 

KeyboardInterrupt: 

#### Generator is a shortcut for defining Iteration

* It uses a special keyword **yield**
* **yield** is like a return
    * but it doesn't exit
* a method that has **yield** acts as both iteratable and iterator

In [78]:
def fib(terms=None):
    a=1
    b=1
    print('returning fib(1)')
    yield 1
    print('returning fib(2)')
    yield 1
    count=2
    while terms==None or count<terms:
        c=a+b
        count+=1
        print(f'returning fib({count})={c}')
        yield c
        a=b
        b=c


In [84]:
x=fib(10)
print(type(x))

<class 'generator'>


In [85]:
it=iter(x)
print(it is x)

True


In [86]:
print(next(it))
print(next(it))

returning fib(1)
1
returning fib(2)
1


In [87]:
next(it)

returning fib(3)=2


2

In [88]:
next(it)

returning fib(4)=3


3

#### prime_range

In [90]:
def is_prime(x):
    if x<2:
        return False
    for i in range(2,x):
        if x%i==0:
            return False
    return True

def prime_range(min,max):
    for x in range(min,max):
        if is_prime(x):
            yield x

In [91]:
for prime in prime_range(2,100):
    print(prime, end='\t')

2	3	5	7	11	13	17	19	23	29	31	37	41	43	47	53	59	61	67	71	73	79	83	89	97	